# Tools

This notebook contains all the tools that will be used by the agent.

In [1]:
import requests
import sqlite3
import pandas as pd

from smolagents import tool, Tool

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

## Creating simple tool

A Python function must be annotated with `@tool`. It should also have a docstring describing what does the function do, what does it return and the description of its parameters.

### City to location

The following function look up at latitude and longitude of a city.

In [4]:
# TODO: Load CSV file containing latitude, longitude and altitude of cities
# https://github.com/bahar/WorldCityLocations/tree/master
df = pd.read_csv('/content/data/cities_latlng.csv', sep=";")

df.columns = [ 'id', 'country', 'city', 'latitude', 'longitude', 'altitude']


In [5]:
# TODO: Explore the loaded dataframe
print(df.head())


   id      country            city   latitude  longitude  altitude
0   2  Afghanistan        Kandahar  31.610000  65.699997      1015
1   3  Afghanistan  Mazar-e Sharif  36.706944  67.112221       369
2   4  Afghanistan           Herat  34.340000  62.189999       927
3   5  Afghanistan       Jalalabad  34.420000  70.449997       573
4   6  Afghanistan          Konduz  36.720000  68.860001       394


In [21]:
# TODO: Add tool description

@tool
def get_latlng(city: str) -> any:
  """
  This function returns the latitude, longitude, altitude for a given city.

  Args:
    city: the name of the city that we want to get the latitude, longitude, altitude

  Returns:
    any: a dictionary with the following keys: city, latitude, longitude, altitude

  Example:
    result = get_latlng('singapore')
  """
  r = df.query(f"city.str.lower() == '{city.lower()}'")
  return { 'city': city, 'latitude': float(r.iloc[0]['latitude']), 'longitude': float(r.iloc[0]['longitude']), 'altitude': float(r.iloc[0]['altitude']) }




In [22]:
# TODO: Test get_latlng method
# case insensitive search
get_latlng('tokyo')


{'city': 'tokyo',
 'latitude': 35.6895266,
 'longitude': 139.6916809,
 'altitude': 40.0}

### Temperature at latitude and longitude

The following function lookup the weather at the given latitude and longtude.

In [23]:
# TODO: Add tool description
@tool
def get_temperature(latitude: float, longitude: float) -> any:
   """
   Return the temperature of a city given by a set of latitude and longtude

   Args:
    latitude: latitude of a city
    longitude: longitude of a city

   Return:
    any: a dictionary with the following keys: temperature_unit, temperature

   Example:
    result = get_temperature(35.6895266, 139.6916809)
   """

   url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
   resp = requests.get(url)
   j = resp.json()
   if resp.status_code >= 400:
      raise Exception(j['reason'])
   temperature = j['current']['temperature_2m']
   units = j['current_units']['temperature_2m']
   return { "temperature_unit": units, "temperature": temperature }


In [24]:
# TODO: Test get_temperature method

result = get_temperature(35.6895266, 139.6916809)
print(result)


{'temperature_unit': '°C', 'temperature': 10.4}


In [ ]:
# TODO: Create a web search tool

def web_search(query: str) -> any:

   search = DuckDuckGoSearchRun()
   return search.invoke(query)

In [ ]:
# TODO: Test the web search


### Query relational database

The following function queries a relational database (SQLite) view called `album_track`. The table's schema is as follows:
| Field name  | Type          |
|-------------|---------------|
| AlbumId     | integer       |
| Title       | nvarchar(160) |
| track_name  | nvarchar(200) |
| artist_name | nvarchar(120) |
| duration    | integer       |
| composer    | nvarchar(220) |


In [17]:
# TODO: Add tool description

@tool
def query_album_track(query: str) -> str:
  """
  Performs SQL queries on the album_track table. Returns the result as an array of tuples.
  The table name is album_track and has the following columns and the corresponding data type:
    AlbumId: integer
    Title: nvarchar(160)
    track_name: nvarchar(200)
    artist_name:nvarchar(120)
    duration: integer
    composer: nvarchar(220)
  The duration is in milliseconds

  Args:
    query: A valid Sqlite SQL statement

  Returns:
    any: list of tuple. Each element of the tuple correspond to the record returned by the SQL query

  Example:
    results = query_album_track('select * from album_track limit 10')

  """
  database = "/content/data/chinook_sqlite.sqlite"
  conn = sqlite3.connect(database)
  try:
      cursor = conn.cursor()
      rows = cursor.execute(query)
      return rows.fetchall()
  finally:
      conn.close()

In [16]:
# TODO: Test the query_album_track function
query_album_track('select * from album_track limit 5')

[(1,
  'For Those About To Rock We Salute You',
  'For Those About To Rock (We Salute You)',
  'AC/DC',
  343719,
  'Angus Young, Malcolm Young, Brian Johnson'),
 (2, 'Balls to the Wall', 'Balls to the Wall', 'Accept', 342562, None),
 (3,
  'Restless and Wild',
  'Fast As a Shark',
  'Accept',
  230619,
  'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman'),
 (3,
  'Restless and Wild',
  'Restless and Wild',
  'Accept',
  252051,
  'F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman'),
 (3,
  'Restless and Wild',
  'Princess of the Dawn',
  'Accept',
  375418,
  'Deaffy & R.A. Smith-Diesel')]

### Tools with states

The following isn an example of a more complex tool that requires initialisation

In [19]:

class SQLiteTool(Tool):

   name = "query_tracks"
   description = """
    Performs SQL queries on the album_track table. Returns the result as an array of tuples.
    The table name is album_track and has the following columns and the corresponding data type:
      AlbumId: integer
      Title: nvarchar(160)
      track_name: nvarchar(200)
      artist_name:nvarchar(120)
      duration: integer
      composer: nvarchar(220)
    The duration is in milliseconds
   """
   inputs = {
       "query": {
           "type": "string",
           "description": "A valid Sqlite SQL statement"
       }
   }
   output_type = "any"

   def __init__(self, db_file):
      self.db_file = db_file
      self.setup()

   def setup(self):
      super().setup()
      self.conn = sqlite3.connect(self.db_file)

   def forward(self, query: str) -> str:
      try:
         cursor = self.conn.cursor()
         rows = cursor.execute(query)
         return rows.fetchall()
      except Exception as e:
         print(f'Query exception: {e}')